In [29]:
%pip install cartoframes

     |████████████████████████████████| 245 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 235 kB 2.7 MB/s eta 0:00:01
  Created wheel for carto: filename=carto-1.11.3-py3-none-any.whl size=35073 sha256=ca68d4ba7d749f0c700f72698d88cb06d9c3fc440fd97f08ba3c5d95578f112e
  Stored in directory: /home/cleonortiz/.cache/pip/wheels/6b/a3/41/90fa4334cd280f91d17226f36db7a34b12902f7e5b281afb84
  Created wheel for pyrestcli: filename=pyrestcli-0.6.11-py3-none-any.whl size=8499 sha256=e9338af2e9ecd7bebb0dc9e79309355dfd57bdc41990fe2858d667fd961018b6
  Stored in directory: /home/cleonortiz/.cache/pip/wheels/92/fc/98/2e46573b8e1bbe953345578314c765d3ecdc9532450bb90d79
Successfully built carto pyrestcli
Note: you may need to restart the kernel to use updated packages.


In [40]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import pandas as pd
import sys
sys.path.append('/mnt/c/Users/cleon/Documents/CAL_CAL/ironhack/Proyectos/Headquarters-Hunt/src')
import apifunctions as af
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import shapely.geometry

In [2]:
sim_comp = pd.read_csv('../data/similarcompanies.csv')

In [60]:
client = MongoClient("localhost:27017")
db = client.get_database("Headquarters")


vanc = db.get_collection("Vancouver")
bev = db.get_collection("BeverlyHills")
irv = db.get_collection('Irvine')

In [61]:
geovan = pd.DataFrame(list(vanc.find()))
geobeb = pd.DataFrame(list(bev.find()))
geoirv = pd.DataFrame(list(irv.find()))

In [62]:
geovan.sample()

,_id,nombre,latitud,longitud,category,distance,location
16,6192581bee1960c50aeaf3a7,Spot Solutions,49.284899,-123.094127,Tech Startup,1935,"{'type': 'Point', 'coordinates': [49.284899, -..."


In [66]:
gdfvan = gpd.GeoDataFrame(geovan, geometry=gpd.points_from_xy(geovan.longitud, geovan.latitud))
gdfvan.drop('location', axis=1,inplace=True)
gdfvan

,_id,nombre,latitud,longitud,category,distance,geometry
0,6192581bee1960c50aeaf397,Marketing.AI,49.278876,-123.119419,Tech Startup,319,POINT (-123.11942 49.27888)
1,6192581bee1960c50aeaf398,Anemone Hug Interactive,49.283573,-123.112627,Tech Startup,874,POINT (-123.11263 49.28357)
2,6192581bee1960c50aeaf399,Rx Networks Inc.,49.288435,-123.123421,Tech Startup,1416,POINT (-123.12342 49.28844)
3,6192581bee1960c50aeaf39a,Bench HQ,49.282817,-123.103898,Tech Startup,1213,POINT (-123.10390 49.28282)
4,6192581bee1960c50aeaf39b,Hootsuite Media,49.264233,-123.104446,Tech Startup,1641,POINT (-123.10445 49.26423)
...,...,...,...,...,...,...,...
144,6192581bee1960c50aeaf427,Starbucks,49.282568,-123.122399,Coffee Shop,784,POINT (-123.12240 49.28257)
145,6192581bee1960c50aeaf428,Starbucks,49.284634,-123.124640,Coffee Shop,1064,POINT (-123.12464 49.28463)
146,6192581bee1960c50aeaf429,Starbucks,49.281780,-123.124590,Coffee Shop,802,POINT (-123.12459 49.28178)
147,6192581bee1960c50aeaf42a,Starbucks,49.286491,-123.121309,Coffee Shop,1165,POINT (-123.12131 49.28649)


In [64]:
gdfvan['geometry']=gdfvan['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdfvan_dic = gdfvan.to_dict(orient="records")
vanc.create_index([("geometry", GEOSPHERE)])
vanc.insert_many(gdfvan_dic)

/home/cleonortiz/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6192581bee1960c50aeaf397')}, 'errmsg': "E11000 duplicate key error collection: Headquarters.Vancouver index: _id_ dup key: { _id: ObjectId('6192581bee1960c50aeaf397') }", 'op': {'_id': ObjectId('6192581bee1960c50aeaf397'), 'nombre': 'Marketing.AI', 'latitud': 49.278876040017614, 'longitud': -123.1194186747484, 'category': 'Tech Startup', 'distance': 319, 'geometry': {'type': 'Point', 'coordinates': (-123.1194186747484, 49.278876040017614)}}}], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}

In [20]:
db.Vancouver.create_index([("location", GEOSPHERE)])
# db.bev.create_index([("location", GEOSPHERE)])
# db.irv.create_index([("location", GEOSPHERE)])

OperationFailure: Index build failed: 2463dde4-f80c-46c9-9523-7bec93ad9224: Collection Headquarters.Vancouver ( c58f2c2a-0b82-4baf-9101-b6e51db7fbe0 ) :: caused by :: Can't extract geo keys: { _id: ObjectId('6192581bee1960c50aeaf397'), nombre: "Marketing.AI", latitud: 49.27887604001761, longitud: -123.1194186747484, category: "Tech Startup", distance: 319, location: { type: "Point", coordinates: [ 49.27887604001761, -123.1194186747484 ] } }  longitude/latitude is out of bounds, lng: 49.2789 lat: -123.119, full error: {'ok': 0.0, 'errmsg': 'Index build failed: 2463dde4-f80c-46c9-9523-7bec93ad9224: Collection Headquarters.Vancouver ( c58f2c2a-0b82-4baf-9101-b6e51db7fbe0 ) :: caused by :: Can\'t extract geo keys: { _id: ObjectId(\'6192581bee1960c50aeaf397\'), nombre: "Marketing.AI", latitud: 49.27887604001761, longitud: -123.1194186747484, category: "Tech Startup", distance: 319, location: { type: "Point", coordinates: [ 49.27887604001761, -123.1194186747484 ] } }  longitude/latitude is o

In [56]:
coord = af.type_point([sim_comp['longitude'][0], sim_comp['latitude'][0]])


In [57]:
# query = {"location": {"$near": {"$geometry": coord, "$maxDistance": 5000}}}
list(vanc.find(query))

OperationFailure: error processing query: ns=Headquarters.VancouverTree: GEONEAR  field=location maxdist=5000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error :: caused by :: unable to find index for $geoNear query, full error: {'ok': 0.0, 'errmsg': 'error processing query: ns=Headquarters.VancouverTree: GEONEAR  field=location maxdist=5000 isNearSphere=0\nSort: {}\nProj: {}\n planner returned error :: caused by :: unable to find index for $geoNear query', 'code': 291, 'codeName': 'NoQueryExecutionPlans'}

In [ ]:
query = {'distance' : {'$lte': 5000}}
list(irv.find(query).sort('distance'))